### Ce notebook présente la création d'un système de géoréférencement à l'aide du package `georefcam`. Il utilise en particulier un DEM issu du package `elevation` et un modèle de caméra ponctuelle.

In [19]:
%matplotlib widget
import geopandas as gpd
import logging
import math
import numpy as np
import pickle
import pandas as pd
import pyvista as pv

from georefcam.camera_model import SimpleCameraModel
from georefcam.dem import EioDEM
from georefcam.georefcam import GeoRefCam, project_points_to_crs
from pathlib import Path
from PIL import Image

# logging.basicConfig(level=logging.DEBUG)

# Définition des dossiers contenant les données nécessaires à l'exécution du notebook (À ÉDITER)

In [20]:
camdata_dir = Path("../../data/pyronear_cam_data")
terrain_dir = "../../data/terrain/"
cam_name = "brison_4"
test_img_name = "pyronear_brison_4_2023_07_04T06_07_57.jpg"
gcp_filepath = "../../georefcam/df_annotations.csv"

# Récupération des données de localisation/orientation de la caméra

Création d'un geodataframe des caméras utilisées à l'aide du fichier de référence 'API_DATA - devices.csv' :

In [21]:
cam_file = camdata_dir / "API_DATA - devices.csv"
df_cams = pd.read_csv(cam_file)
gdf_cams = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df_cams.lon, df_cams.lat, crs="WGS84"), data=df_cams)

Sélection d'une caméra et récupération de ses informations

In [22]:
cam_info = gdf_cams[gdf_cams.login == cam_name].squeeze()
test_img_path = camdata_dir / cam_name / test_img_name
test_img = Image.open(test_img_path)
test_img_res = test_img.size

# Création d'une caméra géoréférencée avec `georefcam`

Détermination des limites géographiques du dem (centré sur la caméra, 50km de côté)

In [23]:
def bounds_from_distance(lat, lon, km):
    dlat = (km/2) / 111.11
    dlon = dlat / math.cos(math.radians(lat))
    return [lon - dlon, lat - dlat, lon + dlon, lat + dlat]

bounds = bounds_from_distance(cam_info.lat, cam_info.lon, 50)

Création d'un DEM avec `EioDEM`

In [ ]:
demfilepath = Path.cwd() / Path(terrain_dir+'eiodem.tiff')
dem = EioDEM(bounds, filepath=demfilepath)
dem.build_pcd(sample_step=1)
dem.build_mesh()

Sauvegarde du DEM en cas de crash du kernel (il peut aussi facilement être recréé à l'aide des cellules précédentes)

In [36]:
with open(terrain_dir+"dem_eio.pkl", "wb") as f:
    pickle.dump(dem, f)

Création d'un modèle de caméra orientée et localisée avec `SimpleCameraModel`

In [37]:
cam_view_x_angle, cam_view_y_angle = 87, 44
cam_roll = 0
cam_model = SimpleCameraModel(
    test_img_res,
    cam_view_x_angle,
    cam_view_y_angle,
    cam_info.azimuth,
    cam_info.pitch,
    cam_roll,
    cam_info.lat,
    cam_info.lon,
    cam_info.elevation,
    dem.crs
)

Création du modèle de caméra géoréférencée

In [38]:
geocam = GeoRefCam(cam_model, dem)

# Évaluation de la projection d'un ensemble de GCP

In [39]:
df_annot_gcp = pd.read_csv(gcp_filepath)
df_annot_gcp = df_annot_gcp.rename({"ele": "alt"}, axis=1).astype({"posx": int, "posy": int, "lat": float, "lon": float, "alt": float})
df_annot_gcp
gdf_annot_gcp = gpd.GeoDataFrame(df_annot_gcp, geometry=gpd.points_from_xy(df_annot_gcp.lon, df_annot_gcp.lat, df_annot_gcp.alt), crs="WGS84")
gdf_annot_gcp

,Unnamed: 0,posx,posy,lat,lon,alt,geometry
0,0,651,328,44.609620,4.098371,1448.0,POINT Z (4.09837 44.60962 1448.00000)
1,1,789,319,44.612899,4.126470,1458.0,POINT Z (4.12647 44.61290 1458.00000)
2,2,978,409,44.577702,4.188103,643.0,POINT Z (4.18810 44.57770 643.00000)
3,3,1190,310,44.612280,4.185187,1316.0,POINT Z (4.18519 44.61228 1316.00000)


Sélection d'un ensemble de GCPs et génération de leurs rayons de projection dans le repère monde

In [40]:
test_gcp_idx = [2, 2]
pixel_points = gdf_annot_gcp.loc[test_gcp_idx, ["posx", "posy"]].values

rays = geocam.camera_model.project_pixel_points_to_world_rays(pixel_points)
rays

array([[[     0.        , -43878.77833555],
        [     0.        ,  89859.07195036],
        [   780.        ,  -4443.91261449]],

       [[     0.        , -43878.77833555],
        [     0.        ,  89859.07195036],
        [   780.        ,  -4443.91261449]]])

Calcul de l'intersection entre les rayons et le DEM

In [41]:
filtered_inter_points, filtered_inter_triangles, df_ray = geocam.cast_rays(
                rays, check_crs=False, min_d_intersection_m=50, return_df_ray=True, cast_method="seq", max_cp_time_s=30)

df_ray

,ori_x,ori_y,ori_z,dest_x,dest_y,dest_z,inter_x,inter_y,inter_z,n_tri,dist_o
n_ray,,,,,,,,,,,
0,0.0,0.0,780.0,-43878.778336,89859.07195,-4443.912614,-2266.987549,4642.549316,510.107697,257149,5173.522838
1,0.0,0.0,780.0,-43878.778336,89859.07195,-4443.912614,-2266.987549,4642.549316,510.107697,257149,5173.522838


Sauvegarde des résultats (si le kernel crashe, charger les pickle dans un autre notebook et y exécuter la cellule de plot)

In [42]:
with open(terrain_dir+"eio_simplecam_proj.pkl", "wb") as f:
    pickle.dump([df_ray, gdf_annot_gcp, test_gcp_idx], f)

In [43]:
with open(terrain_dir+"eio_simplecam_proj.pkl", "rb") as f:
    df_ray, gdf_annot_gcp, test_gcp_idx = pickle.load(f)

Récupération des coordonnées GPS du point d'intersection

In [44]:
project_points_to_crs(df_ray.loc[0, ["inter_x", "inter_y", "inter_z"]].to_numpy(), dem.crs, "wgs84")

array([[ 44.58692937,   4.18798672, 510.10769653]])

Plot de la projection

In [ ]:
plot_margin_m = 1000

target_points = np.array([list(gdf_annot_gcp.to_crs(dem.crs).loc[idx, "geometry"].coords)[0] for idx in [test_gcp_idx[0]]])
origins = df_ray[["ori_x", "ori_y", "ori_z"]].values
inter_points = df_ray[["inter_x", "inter_y", "inter_z"]].values

points_of_interest = np.vstack((target_points, origins, inter_points))
(x_min, y_min, _) = points_of_interest.min(axis=0) - plot_margin_m
(x_max, y_max, _) = points_of_interest.max(axis=0) + plot_margin_m

alts_meshgrid = dem.pcd
zone_mask_idx = np.vstack(((alts_meshgrid[:, :, 0] > x_min) & (alts_meshgrid[:, :, 0] < x_max) & (alts_meshgrid[:, :, 1] > y_min) & (alts_meshgrid[:, :, 1] < y_max)).nonzero())
x_min_idx, y_min_idx = zone_mask_idx.min(axis=1)
x_max_idx, y_max_idx = zone_mask_idx.max(axis=1)
# plot_meshgrid = alts_meshgrid[x_min_idx:x_max_idx, y_min_idx:y_max_idx, :]

# plot_pv_meshgrid = pv.StructuredGrid(*[plot_meshgrid[:, :, i] for i in range(3)])
# plot_pv_meshgrid["alt"] = plot_meshgrid[:, :, 2].ravel(order="F")  # add the altitude as a scalar field in order to plot it as a colormap
plot_pv_meshgrid = dem.mesh

plotter = pv.Plotter()
plotter.add_mesh(plot_pv_meshgrid, smooth_shading=True, specular=0.5, specular_power=15)

bar_height_m = round(max(abs(alts_meshgrid[0, x_max_idx, 0] - alts_meshgrid[0, x_min_idx, 0]), abs(alts_meshgrid[y_max_idx, 0, 1] - alts_meshgrid[y_min_idx, 0, 1])) / 10)

targets_pl = [pv.Line(target_point, target_point + np.array([0, 0, bar_height_m])) for target_point in target_points]
origins_pl = [pv.Line(origin, origin + np.array([0, 0, bar_height_m])) for origin in origins]
inters_pl = [pv.Line(inter_point, inter_point + np.array([0, 0, bar_height_m])) for inter_point in inter_points]
ray_cuts = [pv.Line(origin, inter_point) for origin, inter_point in zip(origins, inter_points)]
sep_lines = [pv.Line([target_point[0], target_point[1], max(target_point[2], inter_point[2])], [inter_point[0], inter_point[1], max(target_point[2], inter_point[2])]) for target_point, inter_point in zip(target_points, inter_points)]
intersections = [pv.PolyData(inter_point) for inter_point in inter_points]

for n_proj in range(len(target_points)):
    plotter.add_mesh(origins_pl[n_proj], color="red", line_width=2, opacity=1, label="Camera plumb line")
    plotter.add_mesh(ray_cuts[n_proj], color="orange", line_width=2, opacity=1, label="Ray segment")
    plotter.add_mesh(inters_pl[n_proj], color="lightgreen", line_width=2, opacity=1, label="Intersection point plumb line")
    plotter.add_mesh(targets_pl[n_proj], color="yellow", line_width=2, opacity=1, label="Target point plumb line")
    plotter.add_mesh(sep_lines[n_proj], color="cyan", line_width=2, opacity=1,
                    label=f"horizontal intersection-target separation: {np.linalg.norm(inter_points[n_proj, :2] - target_points[n_proj, :2]):.1f} m")
    plotter.add_mesh(intersections[n_proj], color="maroon", point_size=5, label="Intersection point")

plotter.add_legend()
plotter.show()